# 🕸️ Yazılım Tedarik Zincirinde Kritiklik Haritalaması
## NPM Ekosisteminde Topolojik Risk Analizi

Bu notebook, NPM paket ekosistemini **kompleks ağ teorisi** perspektifinden analiz eder.

## 🎯 Amaç

NPM paketleri arasındaki bağımlılık ilişkilerini yönlü bir graf olarak modelleyip:
- Kritik altyapı paketlerini tespit etmek
- Topolojik riski ölçmek (in-degree, out-degree, betweenness)
- Kaskad etki potansiyelini değerlendirmek
- Gephi görselleştirme dosyaları üretmek

## 📊 Yöntem

1. **Veri Toplama:** ecosyste.ms API ile Top N en popüler paket
2. **Ağ Kurma:** NetworkX DiGraph (Dependent → Dependency)
3. **Metrik Hesaplama:** Derece ve merkeziyet metrikleri
4. **Risk Skorlama:** Normalize + ağırlıklı kombinasyon
5. **Görselleştirme:** Matplotlib + Gephi export

---

In [ ]:
# Kütüphaneleri içe aktar
from pathlib import Path
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from importlib import reload

# Analysis modülünü yola ekle
sys.path.insert(0, str(Path.cwd().parent))
import analysis.analysis_helpers as ah
from analysis import exporter

# Modülü yeniden yükle (değişiklikleri algılamak için)
ah = reload(ah)

# Parametreler
TOP_N = 1000           # Analiz edilecek paket sayısı (max 2000)
SAMPLE_K = 200         # Betweenness örnekleme boyutu (>1000 düğüm için)
OUTDIR = Path("../results")
OUTDIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Ayarlar yüklendi:")
print(f"   Top N: {TOP_N}")
print(f"   Betweenness örnekleme: {SAMPLE_K}")
print(f"   Çıktı dizini: {OUTDIR}")

## 1. Veri Toplama: Top N Paket Listesi

ecosyste.ms API'den en popüler NPM paketlerini çekiyoruz.

## 2. Top N Paket Listesi

ecosyste.ms API'den en popüler NPM paketlerini çekiyoruz.

In [ ]:
# Top N paket listesini API'den çek
top_packages = ah.fetch_top_packages(TOP_N)

print(f"✅ {len(top_packages)} paket çekildi")
print(f"\n📦 İlk 10 paket:")
for i, pkg in enumerate(top_packages[:10], 1):
    print(f"   {i:2d}. {pkg}")

## 3. Graf Oluşturma: Bağımlılık Ağı

Her paket için dependencies çekip yönlü graf oluşturuyoruz.  
**Kenar yönü:** Dependent → Dependency (bağımlı olan → bağımlılık)

In [ ]:
# Yönlü bağımlılık grafını oluştur
# Sadece Top N paketlerin 1. derece dependencies'ini çek
G, top_set = ah.build_dependency_graph(
    top_packages, 
    cache_path=OUTDIR / "cache_deps.json"
)

print(f"\n✅ Graf oluşturuldu:")
print(f"   Düğüm sayısı: {G.number_of_nodes()}")
print(f"   Kenar sayısı: {G.number_of_edges()}")
print(f"   Ortalama derece: {2 * G.number_of_edges() / G.number_of_nodes():.2f}")

## 4. Merkeziyet Metrikleri

Ağdaki her düğüm için üç temel metrik hesaplıyoruz:

- **In-Degree:** Bu pakete kaç paket bağımlı? (Etki alanı)
- **Out-Degree:** Bu paket kaç pakete bağımlı? (Karmaşıklık)
- **Betweenness:** Ağdaki köprü rolü (En kısa yol aracılığı)

In [ ]:
# Merkeziyet metriklerini hesapla
in_deg, out_deg, btw = ah.compute_metrics(G, sample_k=SAMPLE_K)

print(f"✅ Metrikler hesaplandı:")
print(f"   Ortalama in-degree: {np.mean(list(in_deg.values())):.2f}")
print(f"   Ortalama out-degree: {np.mean(list(out_deg.values())):.2f}")
print(f"   Ortalama betweenness: {np.mean(list(btw.values())):.4f}")
print(f"\n📊 Maksimum değerler:")
print(f"   Max in-degree: {max(in_deg.values())}")
print(f"   Max out-degree: {max(out_deg.values())}")
print(f"   Max betweenness: {max(btw.values()):.4f}")

## 5. Risk Skoru Hesaplama

Normalize edilmiş metriklerin ağırlıklı kombinasyonu:  
**Risk = 0.5 × In-Degree' + 0.2 × Out-Degree' + 0.3 × Betweenness'**

In [ ]:
# Normalize edilmiş metriklere göre risk skoru
risk = ah.compute_risk_scores(in_deg, out_deg, btw)

# En riskli 10 paket
top_risk = sorted(risk.items(), key=lambda kv: kv[1], reverse=True)[:10]
print(f"✅ Risk skorları hesaplandı")
print(f"\n📊 En Riskli 10 Paket:")
for i, (pkg, score) in enumerate(top_risk, 1):
    print(f"{i:2d}. {pkg:30s} Risk: {score:.4f}")

## 6. Sonuçları Kaydet

Tüm analiz sonuçlarını CSV/JSON formatında kaydediyoruz.

In [ ]:
# CSV/JSON dosyalarını kaydet
ah.save_edges(G, OUTDIR / 'edges.csv')
ah.save_metrics(in_deg, out_deg, btw, top_set, OUTDIR / 'metrics.csv')
ah.save_risk_scores(risk, in_deg, out_deg, btw, top_set, OUTDIR / 'risk_scores.csv')
ah.save_graph_stats(G, OUTDIR / 'graph_stats.json')

print(f"✅ Dosyalar kaydedildi:")
print(f"   → {OUTDIR}/edges.csv")
print(f"   → {OUTDIR}/metrics.csv")
print(f"   → {OUTDIR}/risk_scores.csv")
print(f"   → {OUTDIR}/graph_stats.json")

## 7. Gephi Export: Görselleştirme Dosyaları

ID bazlı CSV dosyaları üretiyoruz (Gephi için hazır format).

In [ ]:
# Önce CSV dosyalarını okuyarak metrik dictionary'leri oluştur
import csv

metrics_dict = {}
risks_dict = {}

# metrics.csv'yi oku
metrics_path = OUTDIR / "metrics.csv"
if metrics_path.exists():
    with metrics_path.open(encoding="utf-8") as f:
        for row in csv.DictReader(f):
            pkg = row.get("package", "")
            if pkg:
                metrics_dict[pkg] = row

# risk_scores.csv'yi oku
risk_path = OUTDIR / "risk_scores.csv"
if risk_path.exists():
    with risk_path.open(encoding="utf-8") as f:
        for row in csv.DictReader(f):
            pkg = row.get("package", "")
            if pkg:
                risks_dict[pkg] = row

# Gephi dosyalarını üret
id_map = exporter.export_gephi_from_graph(G, metrics_dict, risks_dict, OUTDIR)

print(f"✅ Gephi dosyaları oluşturuldu:")
print(f"   → {OUTDIR}/gephi_nodes.csv ({len(id_map)} düğüm)")
print(f"   → {OUTDIR}/gephi_edges.csv ({G.number_of_edges()} kenar)")
print(f"\n💡 Gephi'de açmak için:")
print(f"   1. Import spreadsheet → gephi_edges.csv (Edges, Directed)")
print(f"   2. Import spreadsheet → gephi_nodes.csv (Nodes)")

## 8. Görselleştirme: Metrik Dağılımları

In-degree ve out-degree dağılımlarını histogram olarak çizelim.

In [ ]:
# Derece dağılımlarını görselleştir
# Plots klasörünü oluştur
(OUTDIR / 'plots').mkdir(parents=True, exist_ok=True)

in_vals = np.array(list(in_deg.values()))
out_vals = np.array(list(out_deg.values()))

fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)

# In-Degree histogram
axes[0].hist(in_vals, bins=30, color='#3498db', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('In-Degree')
axes[0].set_ylabel('Frekans')
axes[0].set_title('In-Degree Dağılımı')
axes[0].set_yscale('log')
axes[0].grid(alpha=0.3)

# Out-Degree histogram
axes[1].hist(out_vals, bins=30, color='#2ecc71', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Out-Degree')
axes[1].set_ylabel('Frekans')
axes[1].set_title('Out-Degree Dağılımı')
axes[1].set_yscale('log')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(OUTDIR / 'plots' / 'degree_distributions.png', dpi=150, bbox_inches='tight')
plt.savefig(OUTDIR / 'plots' / 'degree_distributions.svg', bbox_inches='tight')
plt.show()

print(f"✅ Dağılım grafikleri kaydedildi")

## 9. Top 10 Liderler

En yüksek merkeziyete sahip paketleri görselleştirelim.

In [ ]:
# En yüksek in-degree'li 10 paket
top10_in = sorted(in_deg.items(), key=lambda x: x[1], reverse=True)[:10]
packages_in = [pkg for pkg, _ in top10_in][::-1]
values_in = [val for _, val in top10_in][::-1]

# En yüksek betweenness'li 10 paket
top10_btw = sorted(btw.items(), key=lambda x: x[1], reverse=True)[:10]
packages_btw = [pkg for pkg, _ in top10_btw][::-1]
values_btw = [val for _, val in top10_btw][::-1]

fig, axes = plt.subplots(1, 2, figsize=(14, 5), dpi=100)

# In-Degree bar chart
axes[0].barh(packages_in, values_in, color='#3498db', edgecolor='black')
axes[0].set_xlabel('In-Degree')
axes[0].set_title('Top 10 In-Degree (En Çok Bağımlılık)')
axes[0].grid(axis='x', alpha=0.3)

# Betweenness bar chart
axes[1].barh(packages_btw, values_btw, color='#e74c3c', edgecolor='black')
axes[1].set_xlabel('Betweenness Centrality')
axes[1].set_title('Top 10 Betweenness (Köprü Paketler)')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTDIR / 'plots' / 'top10_leaders.png', dpi=150, bbox_inches='tight')
plt.savefig(OUTDIR / 'plots' / 'top10_leaders.svg', bbox_inches='tight')
plt.show()

print(f"✅ Top 10 grafikleri kaydedildi")

## 10. Korelasyon Analizi

Metrikler arasındaki ilişkileri scatter plot ile inceleyelim.

In [ ]:
# Metrik korelasyonları
all_packages = list(G.nodes())
in_values = [in_deg.get(pkg, 0) for pkg in all_packages]
out_values = [out_deg.get(pkg, 0) for pkg in all_packages]
btw_values = [btw.get(pkg, 0.0) for pkg in all_packages]

fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100)

# In-Degree vs Betweenness
axes[0].scatter(in_values, btw_values, s=10, alpha=0.5, color='#9b59b6')
axes[0].set_xlabel('In-Degree')
axes[0].set_ylabel('Betweenness')
axes[0].set_title('In-Degree vs Betweenness')
axes[0].grid(alpha=0.3)

# In-Degree vs Out-Degree
axes[1].scatter(in_values, out_values, s=10, alpha=0.5, color='#e67e22')
axes[1].set_xlabel('In-Degree')
axes[1].set_ylabel('Out-Degree')
axes[1].set_title('In-Degree vs Out-Degree')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(OUTDIR / 'plots' / 'metric_correlations.png', dpi=150, bbox_inches='tight')
plt.savefig(OUTDIR / 'plots' / 'metric_correlations.svg', bbox_inches='tight')
plt.show()

print(f"✅ Korelasyon grafikleri kaydedildi")

## 11. Risk Skorları: Top 20

En riskli paketleri görselleştirelim.

In [ ]:
# En riskli 20 paketi göster
top20_risk = sorted(risk.items(), key=lambda x: x[1], reverse=True)[:20]
risk_packages = [pkg for pkg, _ in top20_risk][::-1]
risk_values = [val for _, val in top20_risk][::-1]

plt.figure(figsize=(10, 8), dpi=100)
plt.barh(risk_packages, risk_values, color='#c0392b', edgecolor='black')
plt.xlabel('Risk Skoru')
plt.title('Top 20 En Riskli NPM Paketleri')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig(OUTDIR / 'plots' / 'top20_risk_scores.png', dpi=150, bbox_inches='tight')
plt.savefig(OUTDIR / 'plots' / 'top20_risk_scores.svg', bbox_inches='tight')
plt.show()

print(f"✅ Risk skoru grafikleri kaydedildi")
print(f"\n📊 En Riskli 5 Paket:")
for i, (pkg, score) in enumerate(top20_risk[:5], 1):
    print(f"   {i}. {pkg:30s} - Risk: {score:.4f}")

## 12. Graf İstatistikleri

Ağın genel yapısını özetleyelim.

In [ ]:
# Graf istatistiklerini hesapla
stats = {
    "nodes": G.number_of_nodes(),
    "edges": G.number_of_edges(),
    "avg_in_degree": np.mean(list(in_deg.values())),
    "avg_out_degree": np.mean(list(out_deg.values())),
    "avg_betweenness": np.mean(list(btw.values())),
    "density": nx.density(G),
    "is_connected": nx.is_weakly_connected(G)
}

# JSON olarak kaydet
with (OUTDIR / "graph_stats.json").open("w", encoding="utf-8") as f:
    json.dump(stats, f, indent=2, ensure_ascii=False)

print("📊 Graf İstatistikleri:")
print(f"   Düğüm sayısı: {stats['nodes']}")
print(f"   Kenar sayısı: {stats['edges']}")
print(f"   Yoğunluk (density): {stats['density']:.6f}")
print(f"   Ortalama in-degree: {stats['avg_in_degree']:.2f}")
print(f"   Ortalama out-degree: {stats['avg_out_degree']:.2f}")
print(f"   Zayıf bağlantılı mı? {stats['is_connected']}")
print(f"\n✅ İstatistikler kaydedildi: graph_stats.json")

## 13. Özet: Üretilen Dosyalar

Analiz tamamlandı! Tüm çıktıları kontrol edelim.

In [ ]:
# Üretilen dosyaları listele
print("=" * 70)
print("✅ ANALİZ TAMAMLANDI!")
print("=" * 70)

print("\n📁 Üretilen Dosyalar:")
print("\n🎨 Gephi Görselleştirme:")
print(f"   • {OUTDIR}/gephi_edges.csv")
print(f"   • {OUTDIR}/gephi_nodes.csv")

print("\n📊 Ham Veri:")
print(f"   • {OUTDIR}/edges.csv")
print(f"   • {OUTDIR}/metrics.csv")
print(f"   • {OUTDIR}/risk_scores.csv")
print(f"   • {OUTDIR}/graph_stats.json")

print("\n📈 Görselleştirmeler:")
plots_dir = OUTDIR / "plots"
if plots_dir.exists():
    plot_files = sorted([f.name for f in plots_dir.iterdir() if f.suffix in ['.png', '.svg']])
    for plot in plot_files[:5]:  # İlk 5'ini göster
        print(f"   • plots/{plot}")
    if len(plot_files) > 5:
        print(f"   ... ve {len(plot_files) - 5} dosya daha")

print("\n" + "=" * 70)
print("💡 Sonraki Adımlar:")
print("   1. Gephi'de açın: gephi_edges.csv + gephi_nodes.csv")
print("   2. Layout: Force Atlas 2")
print("   3. Düğüm boyutu: in_degree")
print("   4. Düğüm rengi: risk_score")
print("=" * 70)